In [313]:
import pandas as pd 
from IPython.display import display

In [314]:
stats = pd.read_csv('player_mvp_stats.csv')
stats.head(3)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,...,Pts Max,Share,Team name,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,0,A.C. Green,PF,37,MIA,82,1,17.2,1.8,...,0.0,0.0,Miami Heat,50,32,0.61,6.0,88.9,86.6,1.73
1,1,1,Alonzo Mourning,C,30,MIA,13,3,23.5,5.6,...,0.0,0.0,Miami Heat,50,32,0.61,6.0,88.9,86.6,1.73
2,2,2,Anthony Carter,PG,25,MIA,72,6,22.6,2.7,...,0.0,0.0,Miami Heat,50,32,0.61,6.0,88.9,86.6,1.73


In [315]:
del stats['Unnamed: 0']
del stats ['Unnamed: 0.1']
predictors = stats.columns.difference(['Player','Pos','Tm','Team name','Share', 'Pts Max', 'Pts Won'])

In [316]:
predictors = list(predictors)

In [317]:
train = stats[stats['Year'] < 2022]
test = stats[stats['Year'] == 2022]

In [318]:
test

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team name,W,L,W/L%,GB,PS/G,PA/G,SRS
439,Aaron Gordon,PF,26,DEN,75,75,31.7,5.8,11.1,0.520,...,0.0,0.0,Denver Nuggets,48,34,0.585,1.0,112.7,110.4,2.16
440,Austin Rivers,SG,29,DEN,67,18,22.1,2.2,5.2,0.417,...,0.0,0.0,Denver Nuggets,48,34,0.585,1.0,112.7,110.4,2.16
441,Bol Bol,PF,22,DEN,14,0,5.8,1.1,1.9,0.556,...,0.0,0.0,Denver Nuggets,48,34,0.585,1.0,112.7,110.4,2.16
442,Bones Hyland,SG,21,DEN,69,4,19.0,3.3,8.3,0.403,...,0.0,0.0,Denver Nuggets,48,34,0.585,1.0,112.7,110.4,2.16
443,Bryn Forbes,SG,28,DEN,75,2,17.1,3.1,7.2,0.429,...,0.0,0.0,Denver Nuggets,48,34,0.585,1.0,112.7,110.4,2.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,Micah Potter,C,23,DET,3,0,10.3,1.7,3.7,0.455,...,0.0,0.0,Detroit Pistons,23,59,0.280,28.0,104.8,112.5,-7.36
9004,Rodney McGruder,SG,30,DET,51,2,14.8,1.9,4.4,0.436,...,0.0,0.0,Detroit Pistons,23,59,0.280,28.0,104.8,112.5,-7.36
9005,Saben Lee,PG,22,DET,37,0,16.3,1.9,4.8,0.390,...,0.0,0.0,Detroit Pistons,23,59,0.280,28.0,104.8,112.5,-7.36
9006,Saddiq Bey,SF,22,DET,82,82,33.0,5.5,13.9,0.396,...,0.0,0.0,Detroit Pistons,23,59,0.280,28.0,104.8,112.5,-7.36


#### baseline model

In [319]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [320]:
def add_rank(df):
    df = df.sort_values('Share', ascending=False)
    df['Rk'] = list(range(1,df.shape[0]+1))
    actual = df.sort_values('Share', ascending=False).head(5)
    df = df.sort_values('Predictions', ascending=False)
    df['Pred_rk'] = list(range(1,df.shape[0]+1))
    df['Diff'] = df['Rk'] - df['Pred_rk']
    return df

In [321]:
def cal_ap(df):
    actual = df.sort_values('Share', ascending=False).head(5)
    found = 0
    ps = []
    for index, row in df.iterrows():
        if row['Player'] in actual['Player'].values:
            found += 1
            p = found/row['Pred_rk']
            ps.append(p)
    return sum(ps) / len(ps)

In [322]:
def backtesting(df, reg, years, predictors):
    APs = []
    all_preds = []
    for year in years[5:]:
        
        train = df[df['Year'] < year]
        test = df[df['Year'] == year]

        reg.fit(train[predictors], train['Share'])

        pred = reg.predict(test[predictors])
        pred = pd.DataFrame(pred, columns=['Predictions'], index=test.index)
        pred = pred.sort_values('Predictions', ascending=False)

        combination = pd.concat([test[['Player','Share']], pred], axis=1).sort_values('Share',ascending=False)
        combination = add_rank(combination)
        combination['Test year'] = year
        combination.set_index('Test year', append=True, inplace=True)

        all_preds.append(combination)
        AP = cal_ap(combination)
        APs.append(AP)
    
    mAP = sum(APs) / len(APs)
    all_preds = pd.concat(all_preds)
    return all_preds, mAP

In [323]:
years = list(range(2001,2023))
_,__ = backtesting(stats, reg, years, predictors)
__

0.7234720036925293

In [324]:
def get_top10_year(df_gr, year):
    return _.groupby('Test year').get_group(year).sort_values('Rk', ascending=True).head(10)

In [325]:
get_top10_year(_,2007)

,,Player,Share,Predictions,Rk,Pred_rk,Diff
,Test year,,,,,,
7219,2007,Dirk Nowitzki,0.882,0.108780,1,9,-8
3987,2007,Steve Nash,0.785,0.124470,2,4,-2
830,2007,Kobe Bryant,0.404,0.089863,3,14,-11
7796,2007,Tim Duncan,0.222,0.154143,4,1,3
4292,2007,LeBron James,0.142,0.138901,5,3,2
8480,2007,Tracy McGrady,0.085,0.112486,6,8,-2
5176,2007,Chris Bosh,0.033,0.093614,7,13,-6
4421,2007,Gilbert Arenas,0.024,0.047044,8,36,-28
8583,2007,Kevin Garnett,0.005,0.113963,9,7,2


#### Do some features engineering

In [326]:
stats_ratio = stats[['PTS', 'AST', 'STL', 'BLK', '3P', 'Year']].groupby('Year').apply(lambda x: x/x.mean())

In [327]:
stats_ratio

,PTS,AST,STL,BLK,3P,Year
0,0.576052,0.274186,0.596349,0.228025,0.000000,1.0
1,1.740958,0.493534,0.447262,5.472596,0.000000,1.0
2,0.819274,2.028973,1.490872,0.228025,0.255504,1.0
3,2.060987,1.699950,1.490872,0.684074,0.000000,1.0
4,1.945776,0.658045,1.043611,2.052223,0.000000,1.0
...,...,...,...,...,...,...
10536,0.735752,0.819562,0.479763,1.528302,0.650951,1.0
10537,0.071202,0.000000,0.000000,0.000000,0.130190,1.0
10538,1.281633,0.601012,1.119447,2.547170,0.520761,1.0
10539,0.474679,0.218550,0.319842,1.273585,0.650951,1.0


In [328]:
stats[['PTS_R','AST_R', 'STL_R', 'BLK_R', '3P_R']] = stats_ratio[['PTS', 'AST', 'STL', 'BLK', '3P']]

In [329]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_R,AST_R,STL_R,BLK_R,3P_R
0,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,0.61,6.0,88.9,86.6,1.73,0.576052,0.274186,0.596349,0.228025,0.000000
1,Alonzo Mourning,C,30,MIA,13,3,23.5,5.6,10.8,0.518,...,0.61,6.0,88.9,86.6,1.73,1.740958,0.493534,0.447262,5.472596,0.000000
2,Anthony Carter,PG,25,MIA,72,6,22.6,2.7,6.7,0.406,...,0.61,6.0,88.9,86.6,1.73,0.819274,2.028973,1.490872,0.228025,0.255504
3,Anthony Mason,PF,34,MIA,80,80,40.7,5.8,11.9,0.482,...,0.61,6.0,88.9,86.6,1.73,2.060987,1.699950,1.490872,0.684074,0.000000
4,Brian Grant,C,28,MIA,82,79,33.8,5.9,12.3,0.479,...,0.61,6.0,88.9,86.6,1.73,1.945776,0.658045,1.043611,2.052223,0.000000


In [330]:
predictors += ['PTS_R','AST_R', 'STL_R', 'BLK_R', '3P_R']
_,__ = backtesting(stats, reg, years, predictors)
__

0.7342653267399717

In [331]:
stats['NPos'] = stats['Pos'].astype('category').cat.codes
stats['NTm'] = stats['Tm'].astype('category').cat.codes

In [332]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,PS/G,PA/G,SRS,PTS_R,AST_R,STL_R,BLK_R,3P_R,NPos,NTm
0,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,88.9,86.6,1.73,0.576052,0.274186,0.596349,0.228025,0.000000,2,17
1,Alonzo Mourning,C,30,MIA,13,3,23.5,5.6,10.8,0.518,...,88.9,86.6,1.73,1.740958,0.493534,0.447262,5.472596,0.000000,0,17
2,Anthony Carter,PG,25,MIA,72,6,22.6,2.7,6.7,0.406,...,88.9,86.6,1.73,0.819274,2.028973,1.490872,0.228025,0.255504,5,17
3,Anthony Mason,PF,34,MIA,80,80,40.7,5.8,11.9,0.482,...,88.9,86.6,1.73,2.060987,1.699950,1.490872,0.684074,0.000000,2,17
4,Brian Grant,C,28,MIA,82,79,33.8,5.9,12.3,0.479,...,88.9,86.6,1.73,1.945776,0.658045,1.043611,2.052223,0.000000,0,17


In [333]:
predictors += ['NPos', 'NTm']
_,__ = backtesting(stats, reg, years, predictors)
__

0.7362968308124018

#### Try random forest

In [334]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500,  random_state=1, min_samples_split=5)
_,__ = backtesting(stats, rf, years, predictors)
__

0.6991142342041892